# Training RL zur Balancierung des Cartpole

Dieses Notebook ist Teil des [AI for Beginners Curriculum](http://aka.ms/ai-beginners). Es wurde inspiriert von [offiziellem PyTorch-Tutorial](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) und [dieser Cartpole-PyTorch-Implementierung](https://github.com/yc930401/Actor-Critic-pytorch).

In diesem Beispiel werden wir RL verwenden, um ein Modell zu trainieren, das eine Stange auf einem Wagen balancieren kann, der sich auf einer horizontalen Skala nach links und rechts bewegen kann. Wir nutzen die [OpenAI Gym](https://www.gymlibrary.ml/)-Umgebung, um die Stange zu simulieren.

> **Hinweis**: Du kannst den Code dieser Lektion lokal ausführen (z. B. mit Visual Studio Code), wobei die Simulation in einem neuen Fenster geöffnet wird. Wenn du den Code online ausführst, musst du möglicherweise einige Anpassungen vornehmen, wie [hier](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7) beschrieben.

Wir beginnen damit, sicherzustellen, dass Gym installiert ist:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Nun erstellen wir die CartPole-Umgebung und sehen uns an, wie man damit arbeitet. Eine Umgebung hat die folgenden Eigenschaften:

* **Action space** ist die Menge der möglichen Aktionen, die wir bei jedem Schritt der Simulation ausführen können.
* **Observation space** ist der Raum der Beobachtungen, die wir machen können.


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Schauen wir uns an, wie die Simulation funktioniert. Die folgende Schleife führt die Simulation aus, bis `env.step` das Abbruchsignal `done` zurückgibt. Wir werden Aktionen zufällig mit `env.action_space.sample()` auswählen, was bedeutet, dass das Experiment wahrscheinlich sehr schnell scheitern wird (die CartPole-Umgebung endet, wenn die Geschwindigkeit des CartPole, seine Position oder sein Winkel bestimmte Grenzen überschreiten).

> Die Simulation wird in einem neuen Fenster geöffnet. Sie können den Code mehrmals ausführen und beobachten, wie er sich verhält.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Du kannst feststellen, dass die Beobachtungen aus 4 Zahlen bestehen. Diese sind:  
- Position des Wagens  
- Geschwindigkeit des Wagens  
- Winkel der Stange  
- Rotationsgeschwindigkeit der Stange  

`rew` ist die Belohnung, die wir bei jedem Schritt erhalten. Im CartPole-Umfeld erhältst du für jeden Simulationsschritt 1 Punkt als Belohnung, und das Ziel ist es, die Gesamtbelohnung zu maximieren, d.h. die Zeit, in der CartPole das Gleichgewicht halten kann, ohne umzufallen.

Während des Reinforcement Learnings ist es unser Ziel, eine **Policy** $\pi$ zu trainieren, die uns für jeden Zustand $s$ sagt, welche Aktion $a$ wir ausführen sollen, also im Wesentlichen $a = \pi(s)$.

Wenn du eine probabilistische Lösung möchtest, kannst du dir die Policy so vorstellen, dass sie eine Menge von Wahrscheinlichkeiten für jede Aktion zurückgibt, d.h. $\pi(a|s)$ würde die Wahrscheinlichkeit bedeuten, dass wir die Aktion $a$ im Zustand $s$ ausführen sollten.

## Policy-Gradient-Methode

Im einfachsten RL-Algorithmus, der **Policy Gradient** genannt wird, trainieren wir ein neuronales Netzwerk, um die nächste Aktion vorherzusagen.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Wir werden das Netzwerk trainieren, indem wir viele Experimente durchführen und unser Netzwerk nach jedem Durchlauf aktualisieren. Lassen Sie uns eine Funktion definieren, die das Experiment ausführt und die Ergebnisse zurückgibt (sogenannte **Spur**) - alle Zustände, Aktionen (und ihre empfohlenen Wahrscheinlichkeiten) und Belohnungen:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Sie können eine Episode mit einem untrainierten Netzwerk ausführen und beobachten, dass die Gesamtbelohnung (auch bekannt als Episodenlänge) sehr niedrig ist:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Einer der kniffligen Aspekte des Policy-Gradient-Algorithmus ist die Verwendung von **abgezinsten Belohnungen**. Die Idee ist, dass wir den Vektor der Gesamtbelohnungen bei jedem Schritt des Spiels berechnen und dabei die frühen Belohnungen mit einem Koeffizienten $gamma$ abdiskontieren. Wir normalisieren auch den resultierenden Vektor, da wir ihn als Gewicht verwenden werden, um unser Training zu beeinflussen:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Jetzt geht's ans eigentliche Training! Wir werden 300 Episoden durchführen, und in jeder Episode werden wir Folgendes tun:

1. Das Experiment ausführen und die Spur aufzeichnen.
2. Die Differenz (`gradients`) zwischen den ausgeführten Aktionen und den vorhergesagten Wahrscheinlichkeiten berechnen. Je geringer die Differenz, desto sicherer können wir sein, dass wir die richtige Aktion gewählt haben.
3. Diskontierte Belohnungen berechnen und die Gradienten mit den diskontierten Belohnungen multiplizieren – das stellt sicher, dass Schritte mit höheren Belohnungen einen größeren Einfluss auf das Endergebnis haben als solche mit niedrigeren Belohnungen.
4. Die erwarteten Zielaktionen für unser neuronales Netzwerk werden teilweise aus den vorhergesagten Wahrscheinlichkeiten während des Laufs und teilweise aus den berechneten Gradienten abgeleitet. Wir verwenden den Parameter `alpha`, um zu bestimmen, in welchem Maße Gradienten und Belohnungen berücksichtigt werden – dies wird als *Lernrate* des Verstärkungsalgorithmus bezeichnet.
5. Schließlich trainieren wir unser Netzwerk mit den Zuständen und den erwarteten Aktionen und wiederholen den Prozess.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Lassen Sie uns nun die Episode mit Rendering ausführen, um das Ergebnis zu sehen:


In [ ]:
_ = run_episode(render=True)

Hoffentlich kannst du sehen, dass der Stab jetzt ziemlich gut balancieren kann!

## Actor-Critic-Modell

Das Actor-Critic-Modell ist eine Weiterentwicklung der Policy-Gradient-Methoden, bei der wir ein neuronales Netzwerk erstellen, das sowohl die Policy als auch die geschätzten Belohnungen lernt. Das Netzwerk wird zwei Ausgaben haben (oder man kann es als zwei separate Netzwerke betrachten):
* **Actor** wird die Aktion empfehlen, die ausgeführt werden soll, indem es uns die Zustandswahrscheinlichkeitsverteilung gibt, wie im Policy-Gradient-Modell.
* **Critic** würde schätzen, wie die Belohnung aus diesen Aktionen aussehen könnte. Es gibt die insgesamt geschätzten zukünftigen Belohnungen im gegebenen Zustand zurück.

Lass uns ein solches Modell definieren:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Wir müssten unsere Funktionen `discounted_rewards` und `run_episode` leicht modifizieren:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Jetzt werden wir die Haupttrainingsschleife ausführen. Wir verwenden den manuellen Netzwerk-Trainingsprozess, indem wir geeignete Verlustfunktionen berechnen und Netzwerkparameter aktualisieren:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Erkenntnisse

Wir haben in dieser Demo zwei RL-Algorithmen kennengelernt: den einfachen Policy-Gradient-Algorithmus und den anspruchsvolleren Actor-Critic-Algorithmus. Sie können sehen, dass diese Algorithmen mit abstrakten Konzepten wie Zustand, Aktion und Belohnung arbeiten – daher können sie auf sehr unterschiedliche Umgebungen angewendet werden.

Reinforcement Learning ermöglicht es uns, die beste Strategie zur Lösung eines Problems allein durch die Betrachtung der endgültigen Belohnung zu erlernen. Die Tatsache, dass wir keine gelabelten Datensätze benötigen, erlaubt es uns, Simulationen viele Male zu wiederholen, um unsere Modelle zu optimieren. Dennoch gibt es immer noch viele Herausforderungen im Bereich RL, die Sie kennenlernen können, wenn Sie sich entscheiden, sich intensiver mit diesem faszinierenden Bereich der KI zu beschäftigen.



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
